In [8]:
import pandas as pd
from commcare_export.checkpoint import CheckpointManagerWithDetails
from commcare_export.commcare_hq_client import CommCareHqClient, AUTH_MODE_APIKEY
from commcare_export.commcare_minilinq import get_paginator, PaginationMode
from datetime import datetime
import re
import  numpy as np
# CommCare credentials
username = 'amutenha@zimttech.org'  # Replace with your actual CommCare username
domain = 'zdip-qa'  # Your CommCare project domain
hq_host = 'https://zdip.zimttech.org'  # Your CommCare HQ custom host
API_KEY = '92fbb3f98433a6929a33015273ebe7b38b6dac39'  # Replace with the API key generated in CommCare


zazicSites= pd.read_excel('zazicFacilities.xlsx')



data = pd.ExcelFile('ZDIPlookup.xlsx')

iqaLookup = pd.read_excel('iqaLookup.xlsx')

# lookupTable = pd.read_excel(data,'previousVMMCSubmissions')

# lookupTable.columns = [col.replace('field: ', '') if col.startswith('field: ') else col for col in lookupTable.columns]

# Initialize API client
api_client = CommCareHqClient(hq_host, domain, username, API_KEY, AUTH_MODE_APIKEY)

# Create a paginator for the 'form' resource (fetching data from a form)
form_paginator = get_paginator(resource='form', pagination_mode=PaginationMode.date_modified, page_size=100)
form_paginator.init()

# Initialize checkpoint manager (None used here for simplicity)
checkpoint_manager = CheckpointManagerWithDetails(None, None, PaginationMode.date_modified)

# Date range for filtering
start_date = datetime(2025,6,30)  # 2 December 2024
end_date = datetime(2025, 12,30)  # 6 December 2024

# Specify the app_id to filter forms
desired_app_id ='0778c330a59625e988e36d5a7d0386b6'  


referrals_data = []
AEs_data = []
general_data = []

# Fetch forms iteratively
# Fetch forms using manual pagination
all_forms = []
offset = 0

params = {
    'limit': 100000,
    'app_id': desired_app_id,
    'received_on_start': start_date.strftime('%Y-%m-%dT%H:%M:%S'),
    'received_on_end': end_date.strftime('%Y-%m-%dT%H:%M:%S')
}

forms = api_client.iterate(
    'form',
    form_paginator,
    checkpoint_manager=checkpoint_manager,
    params=params
)

for form in forms:
    all_forms.append(form)
    if len(all_forms) >= 100000:  # Early stopping if desired
        break

In [9]:
# Print a few fetched forms to verify filtering
for form in all_forms[:5]:  
    pass
    # print(f"Received On: {form.get('received_on')}, App ID: {form.get('app_id')}")

filteredResults =[]
for i in all_forms:
    if i["app_id"] ==desired_app_id:
        filteredResults.append(i)

print(filteredResults)
# Filter and categorize forms by app_id and date
for form in forms:
    if form.get('app_id') == desired_app_id:
        # print(form)
        form_received_date = datetime.strptime(form['received_on'], "%Y-%m-%dT%H:%M:%S.%fZ")
        if start_date <= form_received_date <= end_date:
            general_data.append(form)
# Convert each list to a pandas DataFrame
general_df = pd.json_normalize(general_data)
general_df.to_excel('test.xlsx')

general_df.columns = [re.split(r'[./]', col)[-1] for col in general_df.columns]
desiredColumns = [re.split(r'[./]', col)[-1] for col in iqaLookup.columns]
print("desiredcolumns")
print(desiredColumns)

[{'app_id': '0778c330a59625e988e36d5a7d0386b6', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 4602, 'url': 'https://zdip.zimttech.org/a/zdip-qa/api/form_attachment/v1/7fa45961-79d5-48e9-b21e-94f052a27a0c/form.xml'}}, 'build_id': '9d990504928f4439ae5ecdc771350a10', 'domain': 'zdip-qa', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC IQA', '@uiVersion': '1', '@version': '794', '@xmlns': 'http://openrosa.org/formdesigner/72526D77-6087-4854-811D-97DBBA955075', 'activity_and_time_period_under_review': {'section': 'b'}, 'copy-1-of-data_quality_assessment': {'adverseEvents2WT': 'no', 'clientConcerns2WT': 'yes', 'clientSatisfaction2WT': 'yes', 'dataAccuracy2WT': 'yes', 'demographicAnalysis2WT': 'no', 'ehrEnrollmentRate': 'no', 'ehrUsage': 'no', 'electronic_health_record_ehr_-_look_at_total_number_of_clients_circumcised_': 'OK', 'enrollmentRate2WT': 'yes', 'linkages_and_referrals': 'OK', 'look_at_total_number_of_cli

In [10]:
# New columns that should NOT be deleted
columns_to_retain = desiredColumns
# Columns that must be numeric (including the new ones)
numeric_columns = [
    
]

# general_df['AERecordingYear'] = pd.to_numeric(general_df['AERecordingYear'])
# Combine the two lists (retain columns that are in either columns_to_retain or numeric_columns)
# columns_to_keep = list(set(columns_to_retain + numeric_columns))
columns_to_keep = columns_to_retain
# Identify missing columns
missing_columns = [col for col in columns_to_keep if col not in general_df.columns]

In [11]:
# Add missing columns to the DataFrame and assign zero
for col in missing_columns:
    general_df[col] = np.nan
# Drop the columns that are not in the columns_to_keep list
# Sort the columns alphabetically in both DataFrames

# For the 'general_df', sort the columns alphabetically
general_df_sorted = general_df[columns_to_keep]

# For the AE-specific columns, we can do the same
# ae_columns = [
#     'aeComments', 'ae_type_code', 'date_ae_identified', 'AERecordingYear', 'AERecordingSite', 
#     'client_age', 'circumcising_cadre', 'ae_classification', 'AERecordingMonth', 'mcMethod','vmmc_number'
# ]
# general_df_sorted['facility'] = general_df_sorted['facility'].replace('Vere', 'Jompani')

# referrals_columns =['ReferrelRecordingMonth', 
#     'ReferrelRecordingSite', 'ReferrelRecordingYear', 'services_referred','reason_for_referral']

general_df_sorted =  general_df_sorted[:-2]
# other_referrals = []
# ae_data_df_sorted = general_df[ae_columns][sorted(ae_columns)]
# ae_data_df_sorted = ae_data_df_sorted[ae_data_df_sorted['AERecordingMonth'].notna()]

# referralsDF_sorted = general_df[referrals_columns][sorted(referrals_columns)]
# referralsDF_sorted = referralsDF_sorted[referralsDF_sorted['ReferrelRecordingMonth'].notna()]
# general_df_sorted.to_excel("data.xlsx")

In [12]:
general_df_sorted

general_df_sorted.to_excel("relookedData.xlsx")

In [13]:
cols = general_df_sorted.columns.tolist()
seen = {}
new_cols = []
for col in cols:
    if col in seen:
        # If the column name is already seen, we'll try to consolidate
        # or just rename to distinguish if the user wants to inspect them first
        new_col_name = f"{col}_{seen[col]+1}" # Example: 'District_2'
        new_cols.append(new_col_name)
        seen[col] += 1
    else:
        new_cols.append(col)
        seen[col] = 1

general_df_sorted.columns = new_cols

general_df_sorted['submissionMonth'] = general_df_sorted['submissionMonth'].fillna('July')

general_df_sorted['submissionYear'] = general_df_sorted['submissionYear'].fillna(2025)

general_df_sorted = general_df_sorted.rename(columns={'activityType_2': 'activityType'})

# --- Step 2: Consolidate data from truly duplicate-named columns ---
# Now that we have unique names (e.g., 'District', 'District_2'), we can merge their data.
# We'll define a function to consolidate.

def consolidate_columns(df, base_col_name):
    matching_cols = [col for col in df.columns if col.startswith(base_col_name)]

    if len(matching_cols) > 1:
        # Filter out 'facilityComment' if it was inadvertently included due to startsWith
        if base_col_name == 'facility' and 'facilityComment' in matching_cols:
            matching_cols.remove('facilityComment') # Ensure facilityComment isn't consolidated

        if matching_cols: # Check if there are still columns to consolidate after removal
            merged_series = df[matching_cols].stack().groupby(level=0).first()
            df = df.drop(columns=matching_cols)
            df[base_col_name] = merged_series
    return df

# Apply consolidation for the identified duplicate column types
df_cleaned = general_df_sorted.copy() # Work on a copy

df_cleaned = consolidate_columns(df_cleaned, 'section')
df_cleaned = consolidate_columns(df_cleaned, 'District')
df_cleaned = consolidate_columns(df_cleaned, 'facility')
df_cleaned = consolidate_columns(df_cleaned, 'month_year_repeat_group')
df_cleaned = consolidate_columns(df_cleaned, 'username') # Assuming you want to consolidate username as well
df_cleaned = df_cleaned.rename(columns={'darkClerk_1': 'spaceandClientFlow'})
df_cleaned = df_cleaned.rename(columns={'darkClerk_2': 'vmmcDataClerk'})
df_cleaned.loc[df_cleaned['submissionMonth'] == 'September', 'activityType'] = 'IQA'

df_cleaned['activityType'] = df_cleaned['activityType'].fillna('IQA')
# Display the head of the cleaned DataFrame
# iqa = df_cleaned[df_cleaned['activityType'] == 'IQA']
# rbf =df_cleaned[df_cleaned['activityType'] == 'RBF']
df_cleaned.to_excel("data.xlsx")


In [14]:
df_cleaned

,startingMonth,startingYear,endingMonth,endingYear,clientSpaceSize,spaceCleaniness,waitingSpace,examinationroomTemperature,examionationroomLighting,examinationroomSpace,...,comments_and_recommendations_data_triangulation,today,designation__job_title,full_name,organisation,section,District,facility,month_year_repeat_group,username
0,April,2025,July,2025,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,b,GOROMONZI,Chinyika,"[{'calculatedMonth': 'April', 'calculatedYear'...",nmunikira
1,April,2025,August,2025,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,b,GOROMONZI,Makumbe,"[{'calculatedMonth': 'April', 'calculatedYear'...",vmuseza
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,28/08/2025,NaN,NaN,NaN,b,LUPANE,Tshongogwe Mission Hospital,NaN,vmuseza
3,July,2025,July,2025,NaN,NaN,NaN,NaN,NaN,NaN,...,Need to enter July data into DHIS2,28/08/2025,NaN,NaN,NaN,c,LUPANE,Tshongogwe Mission Hospital,NaN,vmuseza
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,28/08/2025,NaN,NaN,NaN,a,LUPANE,Tshongogwe Mission Hospital,NaN,vmuseza
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,d,SANYATI,Rimuka,NaN,fgwenzi
216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,c,MARONDERA,Dombotombo,NaN,tmunyaradzi
217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,d,MARONDERA,Dombotombo,NaN,jgavhera
218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,e,MARONDERA,Dombotombo,NaN,jgavhera
